# Installing, Cloning & Importing

In [ ]:
!pip install --upgrade tensorflow==2.1

In [ ]:
# Update dataset: !git pull
!git clone https://github.com/casperbh96/COVID-19-Detection.git dataset

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2, time
import tensorflow as tf

tf.__version__

# Loading The Datasets

In [ ]:
covid_path = 'dataset/covid_dataset.csv'
covid_image_path = 'dataset/covid_adjusted/'

normal_path = 'dataset/normal_xray_dataset.csv'
normal_image_path = 'dataset/normal_dataset/'

covid_df = pd.read_csv(covid_path, usecols=['filename', 'finding'])
normal_df = pd.read_csv(normal_path, usecols=['filename', 'finding'])

normal_df = normal_df.head(99)

covid_df.head()

In [ ]:
covid_images = []
covid_labels = []

for index, row in covid_df.iterrows():
    filename = row['filename']
    label = row['finding']
    path = covid_image_path + filename

    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    covid_images.append(image)
    covid_labels.append(label)

normal_images = []
normal_labels = []

for index, row in normal_df.iterrows():
    filename = row['filename']
    label = row['finding']
    path = normal_image_path + filename

    # temporary fix while we preprocess ALL the images
    if filename == '4c268764-b5e5-4417-85a3-da52916984d8.jpg':
        break

    image = cv2.imread(path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    
    normal_images.append(image)
    normal_labels.append(label)

# normalize to interval of [0,1]
covid_images = np.array(covid_images) / 255

# normalize to interval of [0,1]
normal_images = np.array(normal_images) / 255

# Splitting Datasets Into Training and Testing

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer
from tensorflow.keras.utils import to_categorical

# split into training(0.7) and testing(0.15) and validation(0.15)
covid_x_train, covid_x_val, covid_y_train, covid_y_val= train_test_split(
    covid_images, covid_labels, test_size=0.3)
covid_x_test, covid_x_val, covid_y_test, covid_y_val= train_test_split(
    covid_x_val, covid_y_val, test_size=0.5)

normal_x_train, normal_x_val, normal_y_train, normal_y_val = train_test_split(
    normal_images, normal_labels, test_size=0.3)
normal_x_test, normal_x_val, normal_y_test, normal_y_val = train_test_split(
    normal_x_val, normal_y_val, test_size=0.5)



X_train = np.concatenate((normal_x_train, covid_x_train), axis=0)
X_val = np.concatenate((normal_x_val, covid_x_val), axis=0)
X_test = np.concatenate((normal_x_test, covid_x_test), axis=0)
y_train = np.concatenate((normal_y_train, covid_y_train), axis=0)
y_val = np.concatenate((normal_y_val, covid_y_val), axis=0)
y_test = np.concatenate((normal_y_test, covid_y_test), axis=0)

# make labels into categories - either 0 or 1
y_train = LabelBinarizer().fit_transform(y_train)
y_train = to_categorical(y_train)

y_val = LabelBinarizer().fit_transform(y_val)
y_val = to_categorical(y_val)

y_test = LabelBinarizer().fit_transform(y_test)
y_test = to_categorical(y_test)

# Modeling With Keras

In [ ]:
import os
from keras.callbacks import *
from clr_callback import *
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import VGG19
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Flatten # Convert pools feature map into this large feature vector
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Dense
from keras.callbacks import ModelCheckpoint
#cycle learning rate

# define the minimum learning rate, maximum learning rate, batch size,
# step size, CLR method, and number of epochs
MIN_LR = 1e-4
MAX_LR = 1e-2
BATCH_SIZE = 32
STEP_SIZE = 8
#CLR_METHOD = "triangular"
CLR_METHOD = "triangular2"
#NUM_EPOCHS = 100

clr = CyclicLR(base_lr=MIN_LR, max_lr=MAX_LR,
              step_size=STEP_SIZE,mode=CLR_METHOD)


In [ ]:
vggModel = VGG19(weights="imagenet", include_top=False,
    input_tensor=Input(shape=X_train.shape[1:]))

outputs = vggModel.output

outputs = Flatten(name="flatten")(outputs)
outputs = Dropout(0.5)(outputs)
outputs = Dense(2, activation="softmax")(outputs)

model = Model(inputs=vggModel.input, outputs=outputs)



for layer in vggModel.layers:
    layer.trainable = False

model.compile(
        loss='categorical_crossentropy', 
        optimizer='adam', 
        metrics=['accuracy']
)

model.summary()

train_aug = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True
)

In [ ]:
history = model.fit(train_aug.flow(X_train, y_train, batch_size=32),
                    validation_data=(X_val, y_val),
                    validation_steps=len(X_val) / 32,
                    steps_per_epoch=len(X_train) / 32,
                    epochs=96,
                    callbacks=[clr],verbose=1)


In [ ]:
 acc = model.evaluate(X_test, y_test, verbose=0)
 print("test acc: "+ str(acc[1]))

In [ ]:
y_pred = model.predict(X_test, batch_size=32)
y_pred_covid = model.predict(covid_x_test, batch_size=32)

In [ ]:
y_test_covid = LabelBinarizer().fit_transform(covid_y_test)
y_test_covid = to_categorical(y_test_covid)

In [ ]:
from sklearn.metrics import classification_report

print(classification_report(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1)))

In [ ]:
# plot the learning rate history
N = np.arange(0, len(clr.history["lr"]))
plt.figure()
plt.plot(N, clr.history["lr"])
plt.title("Cyclical Learning Rate (CLR)")
plt.xlabel("Training Iterations")
plt.ylabel("Learning Rate")

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('classic')

plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])

plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])

plt.show()

In [ ]:
plt.figure(figsize=(10,10))
plt.style.use('classic')

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])

plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')

plt.legend(['Training', 'Testing'])

plt.show()

In [ ]:
from keras.preprocessing import image

test_image = image.load_img('dataset/normal_dataset/0a4d9634-7ee8-4512-ba83-6ff5e352b2c2.jpg', target_size = (224,224))
#test_image = image.load_img('dataset/covid_adjusted/1-s2.0-S0929664620300449-gr2_lrg-a.jpg.jpg', target_size = (224,224))


test_image = image.img_to_array(test_image)
test_image = test_image *1./255

test_image = np.expand_dims(test_image, axis = 0)
test_image = np.vstack([test_image ])
result = model.predict(test_image,batch_size=32)
print(result)

In [ ]:
import matplotlib.image as mpimg

img=mpimg.imread('dataset/normal_dataset/0a4d9634-7ee8-4512-ba83-6ff5e352b2c2.jpg')
#img=mpimg.imread('dataset/covid_adjusted/1-s2.0-S0929664620300449-gr2_lrg-a.jpg.jpg')
imgplot = plt.imshow(img)
plt=plt.title('Chest X-ray ')

In [ ]:
if result[0][0]<result[0][1]:

    prediction = 'normal'
else:
    prediction = 'covid'
    
print("AI's prediction is: "+ prediction)